In [39]:
from asammdf import MDF, Signal
from asammdf.blocks.types import DbcFileType, BusType, StrPath
from pathlib import Path
import requests
from datetime import datetime, timedelta
from time import sleep
import json
from collections.abc import Iterable
from collections import defaultdict

_path = Path('../files/upper')
files = list(_path.rglob('*.[mM][fF]4'))
print(f'Found {len(files)} MF4 files in the current directory and subdirectories.')

Found 225 MF4 files in the current directory and subdirectories.


In [40]:
def get_dbc_files(directory: Path | str) -> list[StrPath]:
    """
    Get all DBC files in the specified directory.
    """

    if not isinstance(directory, Path):
        directory = Path(directory)

    return list(directory.rglob("*.[dD][bB][cC]"))

def get_dbc_dict(directory: Path | str) -> dict[BusType, Iterable[DbcFileType]]:
    """
    Get a dictionary of DBC files in the specified directory.
    This dictionary can be passed directly to extract_bus_logging() in asammdf.
    """

    dbc_files = get_dbc_files(directory)
    return {"CAN": [(file, 0) for file in dbc_files]}


In [24]:
file_dates = {}
def processing(x: int, y: int):
    print(f' => Processing: {100*x/y:>4.1f}%', flush=True, end='\r')

In [25]:
for f in files[:2]:
    mdf = MDF(f, process_bus_logging=False, progress=processing)
    print(f" => Processing {f} complete", flush=True)
    local_date = mdf.start_time.astimezone(datetime.now().astimezone().tzinfo).date()
    local_date_str = local_date.strftime('%Y_%m_%d')
    if local_date_str not in file_dates:
        file_dates[local_date_str] = []
    file_dates[local_date_str].append(mdf)

 => Processing ../files/upper/data/00001085/00000004.MF4 complete
 => Processing ../files/upper/data/00001085/00000005.MF4 complete


In [41]:
concatenated = MDF().concatenate(list(file_dates.values())[0], process_bus_logging=False)
decoded = concatenated.extract_bus_logging(database_files=get_dbc_dict('../files/upper'))

In [46]:
decoded.search('Motor_Speed')

['HYD_Motor_Speed',
 'CAN2.HYD_Motor_Position_Info.HYD_Motor_Speed',
 'HYD_Motor_Position_Info.HYD_Motor_Speed',
 'CP1_Motor_Speed',
 'CAN2.CP1_Motor_Position_Info.CP1_Motor_Speed',
 'CP1_Motor_Position_Info.CP1_Motor_Speed',
 'CP2_Motor_Speed',
 'CAN2.CP2_Motor_Position_Info.CP2_Motor_Speed',
 'CP2_Motor_Position_Info.CP2_Motor_Speed',
 'FAN_Motor_Speed',
 'CAN2.FAN_Motor_Position_Info.FAN_Motor_Speed',
 'FAN_Motor_Position_Info.FAN_Motor_Speed']

In [48]:
sig = decoded.get('HYD_Motor_Speed')

In [71]:
metric_name = sig.name.replace(" ", "_")
start_ts = mdf.start_time + timedelta(seconds=sig.timestamps[0])
end_ts = mdf.start_time + timedelta(seconds=sig.timestamps[-1])

params = {
    "query": '{__name__!=""}',
    "start": str(start_ts.timestamp()),
    "end": str(end_ts.timestamp()),
    "step": str((sig.timestamps[-1] - sig.timestamps[0]) / 2),
}
print(params)
resp = requests.get('http://localhost:8428/api/v1/query_range', params=params, timeout=10)
resp.text

{'query': '{__name__!=""}', 'start': '1751480983.9905', 'end': '1751481518.1418', 'step': '267.07565'}


'{"status":"success","data":{"resultType":"matrix","result":[]},"stats":{"seriesFetched": "0","executionTimeMsec":2}}'